In [1]:
import pickle

In [2]:
x = pickle.load(open('./archive/x.pkl','rb'))
y = pickle.load(open('./archive/y.pkl','rb'))


In [3]:
x = x.astype('float32') / 255.0


In [4]:
# x = x/255

In [5]:
x.shape

(18000, 50, 50, 3)

In [6]:
y.shape

(18000,)

In [7]:
y[2]

1

In [8]:
from keras import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense


2024-05-02 10:41:32.934937: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-02 10:41:39.657313: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-02 10:41:39.664704: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-02 10:41:56.837592: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [9]:
model = Sequential()

model.add(Conv2D(64,(3,3),activation = 'relu'))
model.add(MaxPooling2D((2,2)))

model.add(Conv2D(64,(3,3),activation = 'relu'))
model.add(MaxPooling2D((2,2)))

model.add(Flatten())

model.add(Dense(128,input_shape = x.shape[1:],activation = 'relu'))

model.add(Dense(20,activation='softmax'))

In [10]:
model.compile(optimizer = 'adam',  loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [11]:
model.fit(x,y,epochs=50,validation_split=0.1)
model.save("./archive/model.h5")

2024-05-02 10:42:26.078256: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 486000000 exceeds 10% of free system memory.


Epoch 1/50


2024-05-02 10:42:36.234647: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 18874368 exceeds 10% of free system memory.
2024-05-02 10:42:40.640471: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 25648128 exceeds 10% of free system memory.
2024-05-02 10:42:40.791442: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 18874368 exceeds 10% of free system memory.


  2/507 [..............................] - ETA: 2:50 - loss: 3.0362 - accuracy: 0.1094   

2024-05-02 10:42:41.338319: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 18874368 exceeds 10% of free system memory.


507/507 [==============================] - 97s 173ms/step - loss: 0.0551 - accuracy: 0.9867 - val_loss: 0.0021 - val_accuracy: 0.9994
Epoch 2/50
507/507 [==============================] - 108s 212ms/step - loss: 1.3292e-05 - accuracy: 1.0000 - val_loss: 0.0013 - val_accuracy: 0.9994
Epoch 3/50
507/507 [==============================] - 83s 164ms/step - loss: 4.4136e-06 - accuracy: 1.0000 - val_loss: 0.0016 - val_accuracy: 0.9994
Epoch 4/50
507/507 [==============================] - 88s 172ms/step - loss: 2.1190e-06 - accuracy: 1.0000 - val_loss: 0.0014 - val_accuracy: 0.9994
Epoch 5/50
507/507 [==============================] - 81s 160ms/step - loss: 1.2301e-06 - accuracy: 1.0000 - val_loss: 0.0015 - val_accuracy: 0.9994
Epoch 6/50
507/507 [==============================] - 81s 160ms/step - loss: 7.6551e-07 - accuracy: 1.0000 - val_loss: 0.0015 - val_accuracy: 0.9994
Epoch 7/50
507/507 [==============================] - 81s 159ms/step - loss: 5.0564e-07 - accuracy: 1.0000 - val_loss: 0

In [4]:

import cv2
import numpy as np

cap = cv2.VideoCapture(0)

cap.set(cv2.CAP_PROP_FRAME_WIDTH,1000) 
cap.set(cv2.CAP_PROP_FRAME_HEIGHT,480)

aweight = 0.5
num_frames = 0
bg = None


def run_avg(img,aweight):
    global bg
    if bg is None:
        bg = img.copy().astype('float')
        return
    cv2.accumulateWeighted(img,bg,aweight)
def segment(img,thres=25):
    global bg
    diff = cv2.absdiff(bg.astype('uint8'),img)
    _, thresholded = cv2.threshold(diff,thres,255,cv2.THRESH_BINARY)
    contours,_ = cv2.findContours(thresholded.copy(),cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    if len(contours) == 0:
        return
    else:
        segmented = max(contours,key = cv2.contourArea)
    return (thresholded,segmented)

flag = 0
i = 0
N1 = 0
while(cap.isOpened()):
    ret, frame = cap.read()

    if ret ==True:
        frame = cv2.flip(frame, 1)
        clone = frame.copy()
        (height, width) = frame.shape[:2]
        roi = frame[100:300, 300:500]
        gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
        gray = cv2.GaussianBlur(gray, (7, 7), 0)

        if num_frames < 30:
            run_avg(gray, aweight)
        else:
            hand = segment(gray)

            if hand is not None:
                (thresholded, segmented) = hand
                cv2.drawContours(clone, [segmented + (300, 100)], -1, (0, 0, 255))
                cv2.imshow("Thesholded", thresholded)
                contours, _= cv2.findContours(thresholded,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE)
                #cv2.drawContours(clone,contours,-1,(0,255,0),4)

                for cnt in contours:
                    area = cv2.contourArea(cnt)
                    #print(area)
                    if area >5000:
                        x,y,w,h = cv2.boundingRect(cnt)
                        #cv2.rectangle(clone,(x,y),(x+w,y+h),(0,255,0),4)
                        print("Hand is detected")
                        to_save = cv2.resize(thresholded,(64,64))
                        cv2.imwrite('./explo_prediction/img'+str(i)+'.jpg',to_save)
                        i = i + 1
        cv2.rectangle(frame, (300, 100), (500, 300), (0, 255, 0), 2)
        cv2.rectangle(clone, (300, 100), (500, 300), (0, 255, 0), 2)
        cv2.putText(clone, "Gesture Controlled Calculator", (50, 50), cv2.FONT_HERSHEY_DUPLEX, 1, (255, 0, 0), 3)

        num_frames += 1

        cv2.imshow('frame', clone)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    else:
        break

cap.release()
cv2.destroyAllWindows()

QObject::moveToThread: Current thread (0x257b0e0) is not the object's thread (0x25363b0).
Cannot move to target thread (0x257b0e0)

QObject::moveToThread: Current thread (0x257b0e0) is not the object's thread (0x25363b0).
Cannot move to target thread (0x257b0e0)

QObject::moveToThread: Current thread (0x257b0e0) is not the object's thread (0x25363b0).
Cannot move to target thread (0x257b0e0)

QObject::moveToThread: Current thread (0x257b0e0) is not the object's thread (0x25363b0).
Cannot move to target thread (0x257b0e0)

QObject::moveToThread: Current thread (0x257b0e0) is not the object's thread (0x25363b0).
Cannot move to target thread (0x257b0e0)

QObject::moveToThread: Current thread (0x257b0e0) is not the object's thread (0x25363b0).
Cannot move to target thread (0x257b0e0)

QObject::moveToThread: Current thread (0x257b0e0) is not the object's thread (0x25363b0).
Cannot move to target thread (0x257b0e0)

QObject::moveToThread: Current thread (0x257b0e0) is not the object's thread

QObject::moveToThread: Current thread (0x257b0e0) is not the object's thread (0x25363b0).
Cannot move to target thread (0x257b0e0)

QObject::moveToThread: Current thread (0x257b0e0) is not the object's thread (0x25363b0).
Cannot move to target thread (0x257b0e0)

QObject::moveToThread: Current thread (0x257b0e0) is not the object's thread (0x25363b0).
Cannot move to target thread (0x257b0e0)

QObject::moveToThread: Current thread (0x257b0e0) is not the object's thread (0x25363b0).
Cannot move to target thread (0x257b0e0)

QObject::moveToThread: Current thread (0x257b0e0) is not the object's thread (0x25363b0).
Cannot move to target thread (0x257b0e0)

QObject::moveToThread: Current thread (0x257b0e0) is not the object's thread (0x25363b0).
Cannot move to target thread (0x257b0e0)

QObject::moveToThread: Current thread (0x257b0e0) is not the object's thread (0x25363b0).
Cannot move to target thread (0x257b0e0)

QObject::moveToThread: Current thread (0x257b0e0) is not the object's thread

Hand is detected
Hand is detected
Hand is detected
Hand is detected
Hand is detected
Hand is detected
Hand is detected
Hand is detected
Hand is detected
Hand is detected
Hand is detected
Hand is detected
Hand is detected
Hand is detected
Hand is detected
Hand is detected
Hand is detected
Hand is detected
Hand is detected
Hand is detected
Hand is detected
Hand is detected
Hand is detected
Hand is detected
Hand is detected
Hand is detected
Hand is detected
Hand is detected
Hand is detected
Hand is detected
Hand is detected
Hand is detected
Hand is detected
Hand is detected
Hand is detected
Hand is detected
Hand is detected
Hand is detected
Hand is detected
Hand is detected
Hand is detected
Hand is detected
Hand is detected
Hand is detected
Hand is detected
Hand is detected
Hand is detected
Hand is detected
Hand is detected
Hand is detected
Hand is detected
Hand is detected
Hand is detected
Hand is detected
Hand is detected
Hand is detected
Hand is detected
Hand is detected
Hand is detect

In [5]:
import cv2
# Assuming 'frame' is your captured image
cv2.imwrite('./archive/image.jpg', thresholded) 


True

In [7]:
import cv2
import numpy as np
from keras.models import load_model

# Load the trained model
model = load_model('./archive/model.h5')

# Preprocess the input image(s)
def preprocess_image(image_path, img_size=50):
    img_arr = cv2.imread(image_path)
    img_arr = cv2.resize(img_arr, (img_size, img_size))
    img_arr = np.expand_dims(img_arr, axis=0)  # Add batch dimension
    img_arr = img_arr / 255.0  # Normalize pixel values
    return img_arr

# Predict function
def predict_image(image_path, model, class_mapping):
    preprocessed_img = preprocess_image(image_path)
    prediction = model.predict(preprocessed_img)
    # Get the predicted class index
    predicted_class_index = np.argmax(prediction)
    # Map the index to the actual class label
    predicted_class_label = class_mapping[predicted_class_index]
    return predicted_class_label

# Define your class mapping
class_mapping = {0: "zero", 1: "one", 2: "two", 3: "three", 4: "four",5: "five", 6: "six", 7: "seven", 8: "eight", 9: "nine",10: "+", 11: "-", 12: "*", 13: "/", 14: "confirm", 15: 'reset', 16: "result", 17:"**", 18:"%", 19:"clear"}

# Example usage
image_path ='./archive/image.jpg'
predicted_class = predict_image(image_path, model, class_mapping)
print("Predicted class:", predicted_class)


1/1 [==============================] - 0s 287ms/step
Predicted class: two
